In [12]:
import cv2
import mediapipe as mp
import os

# Face mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(
    static_image_mode=True,  
    max_num_faces=1,
    refine_landmarks=False,  
    min_detection_confidence=0.5)

# 이미지 파일이 저장된 폴더 경로
image_folder = 'img'  # 이미지가 저장된 폴더 경로를 입력하세요
output_folder = 'crop_img'  # 크롭된 이미지를 저장할 폴더

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 좌표를 원래 크기로 변환하는 함수
def invert_normalization(x, y, w, h):
    return int(x * w), int(y * h)

# 이미지 파일 처리
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # 이미지 파일 형식에 따라 확장자 추가
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"{filename} 파일을 열 수 없습니다.")
            continue

        # BGR 이미지를 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Face mesh를 사용해 얼굴 탐지
        results = face_mesh.process(image_rgb)
        if not results.multi_face_landmarks:
            print(f"{filename}: 얼굴을 탐지할 수 없습니다.")
            continue

        # 랜드마크 추출
        landmarks = results.multi_face_landmarks[0].landmark
        image_width = image.shape[1]
        image_height = image.shape[0]

        # 왼쪽 눈 영역 좌표
        left_eye_coords = [landmarks[i] for i in [33, 133, 159, 145]]
        left_x_min = min([lm.x for lm in left_eye_coords])
        left_y_min = min([lm.y for lm in left_eye_coords])
        left_x_max = max([lm.x for lm in left_eye_coords])
        left_y_max = max([lm.y for lm in left_eye_coords])

        left_x_min, left_y_min = invert_normalization(left_x_min, left_y_min, image_width, image_height)
        left_x_max, left_y_max = invert_normalization(left_x_max, left_y_max, image_width, image_height)

        # 오른쪽 눈 영역 좌표
        right_eye_coords = [landmarks[i] for i in [362, 263, 386, 374]]
        right_x_min = min([lm.x for lm in right_eye_coords])
        right_y_min = min([lm.y for lm in right_eye_coords])
        right_x_max = max([lm.x for lm in right_eye_coords])
        right_y_max = max([lm.y for lm in right_eye_coords])

        right_x_min, right_y_min = invert_normalization(right_x_min, right_y_min, image_width, image_height)
        right_x_max, right_y_max = invert_normalization(right_x_max, right_y_max, image_width, image_height)

        # 여유 공간 설정 (픽셀 단위)
        padding = 10  # 원하는 여유 공간

        # 왼쪽 눈 크롭 좌표 조정
        left_y_min = max(0, left_y_min - padding)
        left_y_max = min(image_height, left_y_max + padding)
        left_x_min = max(0, left_x_min - padding)
        left_x_max = min(image_width, left_x_max + padding)

        # 오른쪽 눈 크롭 좌표 조정
        right_y_min = max(0, right_y_min - padding)
        right_y_max = min(image_height, right_y_max + padding)
        right_x_min = max(0, right_x_min - padding)
        right_x_max = min(image_width, right_x_max + padding)

        # 왼쪽 눈 크롭
        left_eye_region = image[left_y_min:left_y_max, left_x_min:left_x_max]
        left_eye_resized = cv2.resize(left_eye_region, (64, 64))  # 크기를 원하시는 대로 조정 가능

        # 오른쪽 눈 크롭
        right_eye_region = image[right_y_min:right_y_max, right_x_min:right_x_max]
        right_eye_resized = cv2.resize(right_eye_region, (64, 64))  # 크기를 원하시는 대로 조정 가능

        # 크롭된 이미지 저장
        left_output_path = os.path.join(output_folder, f"left_eye_{filename}")
        right_output_path = os.path.join(output_folder, f"right_eye_{filename}")
        
        cv2.imwrite(left_output_path, left_eye_resized)
        cv2.imwrite(right_output_path, right_eye_resized)

        print(f"{filename}의 왼쪽 눈이 {left_output_path}에 저장되었습니다.")
        print(f"{filename}의 오른쪽 눈이 {right_output_path}에 저장되었습니다.")


img_0_0000.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0000.jpg에 저장되었습니다.
img_0_0000.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0000.jpg에 저장되었습니다.
img_0_0001.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0001.jpg에 저장되었습니다.
img_0_0001.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0001.jpg에 저장되었습니다.
img_0_0002.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0002.jpg에 저장되었습니다.
img_0_0002.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0002.jpg에 저장되었습니다.
img_0_0003.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0003.jpg에 저장되었습니다.
img_0_0003.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0003.jpg에 저장되었습니다.
img_0_0004.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0004.jpg에 저장되었습니다.
img_0_0004.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0004.jpg에 저장되었습니다.
img_0_0005.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0005.jpg에 저장되었습니다.
img_0_0005.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0005.jpg에 저장되었습니다.
img_0_0006.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0006.jpg에 저장되었습니다.
img_0_0006.jpg의 오른쪽 눈이 crop_img\right_eye_img_0_0006.jpg에 저장되었습니다.
img_0_0007.jpg의 왼쪽 눈이 crop_img\left_eye_img_0_0007.jpg에 저장되었습니다.
img_0_0007.